# AI TEXT GENERATION FOR BEGINNERS: CREATING AND FINE TUNING YOUR OWN SINGLISH CHATBOT

# PART 3: TEST YOUR FINE TUNED BOT ON A CHAT APP (COLAB)

Loading the fine tuned model on a chat app is a good way to assess the results. The good folks at Plotly have  released a number of [sample NLP apps](https://community.plotly.com/t/show-tell-dash-transformers-apps-auto-summarization-translator-and-chatbot-are-now-runnable-on-colab/42762) based on the transformers library that would allow us to quickly see how the fine tuned Singlish bot performs.

The code below is based on this [Colab notebook](https://colab.research.google.com/github/plotly/dash-sample-apps/blob/master/apps/dash-chatbot/ColabDemo.ipynb) by [Lu Xing Han](https://github.com/xhlulu) @ Plotly, with minor changes on my part.

# 1. IMPORTS AND CONNECTING TO YOUR GDRIVE/FOLDER

Assuming that the model was fine tuned successfully, let's install the necessary libraries and then connect to your Google Drive account and folder where the required files are located ('output-medium').

In [ ]:
!pip install -q jupyter-dash==0.3.0rc1 dash-bootstrap-components transformers

In [ ]:
import dash
import dash_html_components as html
import dash_core_components as dcc
import dash_bootstrap_components as dbc
import os
import time
import torch

from dash.dependencies import Input, Output, State
from google.colab import drive
from jupyter_dash import JupyterDash
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:
# get credentials to log into your GDrive account

drive.mount('/content/drive/')

In [ ]:
# switch to the right directory where the fine-tuned model/files are located

os.chdir("/content/drive/My Drive/Colab Notebooks")

In [ ]:
# switch to DialoGPT-small if you trained on the smaller model

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Device: {device}")

print("Start loading model...")
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
model = AutoModelForCausalLM.from_pretrained("output-medium") #change the file name if you amended it

# Switch to cuda, eval mode, and FP16 for faster inference
if device == "cuda":
    model = model.half()
model.to(device)
model.eval()

print("Done.")

# 2. DEFINE CHAT APP

In [ ]:
def textbox(text, box="other"):
    style = {
        "max-width": "55%",
        "width": "max-content",
        "padding": "10px 15px",
        "border-radius": "25px",
    }

    if box == "self":
        style["margin-left"] = "auto"
        style["margin-right"] = 0

        color = "primary"
        inverse = True

    elif box == "other":
        style["margin-left"] = 0
        style["margin-right"] = "auto"

        color = "light"
        inverse = False

    else:
        raise ValueError("Incorrect option for `box`.")

    return dbc.Card(text, style=style, body=True, color=color, inverse=inverse)

In [ ]:
conversation = html.Div(
    style={
        "width": "80%",
        "max-width": "800px",
        "height": "70vh",
        "margin": "auto",
        "overflow-y": "auto",
    },
    id="display-conversation",
)

controls = dbc.InputGroup(
    style={"width": "80%", "max-width": "800px", "margin": "auto"},
    children=[
        dbc.Input(id="user-input", placeholder="Write to the chatbot...", type="text"),
        dbc.InputGroupAddon(dbc.Button("Submit", id="submit"), addon_type="append",),
    ],
)


# Define app
app = JupyterDash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server


# Define Layout
app.layout = dbc.Container(
    fluid=True,
    children=[
        html.H1("Singlish Chatbot (with DialoGPT-medium)"),
        html.Hr(),
        dcc.Store(id="store-conversation", data=""),
        conversation,
        controls,
    ],
)

In [ ]:
@app.callback(
    Output("display-conversation", "children"), [Input("store-conversation", "data")]
)
def update_display(chat_history):
    return [
        textbox(x, box="self") if i % 2 == 0 else textbox(x, box="other")
        for i, x in enumerate(chat_history.split(tokenizer.eos_token)[:-1])
    ]


@app.callback(
    [Output("store-conversation", "data"), Output("user-input", "value")],
    [Input("submit", "n_clicks"), Input("user-input", "n_submit")],
    [State("user-input", "value"), State("store-conversation", "data")],
)
def run_chatbot(n_clicks, n_submit, user_input, chat_history):
    if n_clicks == 0:
        return "", ""

    if user_input is None or user_input == "":
        return chat_history, ""

    # # temporary
    # return chat_history + user_input + "<|endoftext|>" + user_input + "<|endoftext|>", ""

    # encode the new user input, add the eos_token and return a tensor in Pytorch
    bot_input_ids = tokenizer.encode(
        chat_history + user_input + tokenizer.eos_token, return_tensors="pt"
    ).to(device)

    # generate a response while limiting the total chat history to 1024 tokens,
    # uncomment some of the parameters or change the 'temperature' to see how results vary
    chat_history_ids = model.generate(
        bot_input_ids, max_length=1024, 
        pad_token_id=tokenizer.eos_token_id,
        #no_repeat_ngram_size=3,       
        #do_sample=True, 
        #top_k=100, 
        #top_p=0.7,
        temperature = 0.8
    )
    chat_history = tokenizer.decode(chat_history_ids[0])

    return chat_history, ""

In [ ]:
# Click on the link to open the app.

app.run_server(mode='external')